In [3]:
import ibis
from boring_semantic_layer.semantic_api.api import to_semantic_table

con = ibis.duckdb.connect(":memory:")

BASE_URL = "https://pub-a45a6a332b4646f2a6f44775695c64df.r2.dev"
tables = {
    "flights_tbl": con.read_parquet(f"{BASE_URL}/flights.parquet"),
    "carriers_tbl": con.read_parquet(f"{BASE_URL}/carriers.parquet"),
}

In [4]:
tables["flights_tbl"]

DatabaseTable: ibis_read_parquet_ymeri6hsyraczeezmkfazpfg2e
  carrier     string
  origin      string
  destination string
  flight_num  string
  flight_time int64
  tail_num    string
  dep_time    timestamp(6)
  arr_time    timestamp(6)
  dep_delay   int64
  arr_delay   int64
  taxi_out    int64
  taxi_in     int64
  distance    int64
  cancelled   string
  diverted    string
  id2         int64

In [10]:
flights_st = (
    to_semantic_table(tables["flights_tbl"])
    .with_dimensions(
        origin=lambda t: t.origin,
        carrier=lambda t: t.carrier,
        month=lambda t: t.arr_time.truncate("month"),
    )
    .with_measures(
        flight_count=lambda t: t.count(),
        avg_dep_delay=lambda t: t.dep_delay.mean(),
    )
)

In [2]:
carrier_share = flights_st.group_by("carrier").aggregate(lambda t: t.flight_count)
carrier_expr = carrier_share.to_ibis()
carrier_expr

NameError: name 'flights_st' is not defined

In [14]:
carrier_expr.execute()

OperationNotDefinedError: Compilation rule for 'SemanticTable' operation is not defined